In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# cd /Users/allywang/Desktop/work/S2026/FACT

# python train_cutrun_cnn.py \
#   --fasta /path/to/mm10.fa \
#   --bigwig data/bws_for_predictor/SPT16_4D_2HdTAG_Sox2_S46_1_120_sorted.bw \
#   --chrom chr3 \
#   --window-size 10000 \
#   --stride 10000 \
#   --signal-bins 100 \
#   --batch-size 32 \
#   --epochs 10 \
#   --output-model cutrun_cnn.pt

In [3]:
import sys 
sys.path.append('../src')

from cnn import * 
from datas import * 

In [4]:
bigwig_path = '../data/bws_for_predictor/SPT16_4D_2HdTAG_Sox2_S46_1_120_sorted.bw'
fasta_path = '../data/GRCm38.primary_assembly.genome.fa'
chrom = 'chr3'
window_size = 1000
stride = 1000
signal_bins = 100
batch_size = 256
epochs = 100
lr = 1e-3
device = 'cuda'
weight_decay = 1e-4

In [5]:
dataset = SequenceBigWigDataset(
    fasta_path=str(fasta_path),
    bigwig_path=str(bigwig_path),
    signal_bins=signal_bins,
    chrom=chrom,
    window_size=window_size,
    stride=stride
)

In [6]:
len(dataset)

80346

In [7]:
x, y, y_count = next(iter(dataset))

x.shape, y.shape, y_count

(torch.Size([4, 1000]), torch.Size([100]), tensor(26.2742))

In [ ]:
# λ = (α/2) × n̄^obs    where α < 1
import torch
from torch.utils.data import DataLoader

loader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=False,  # shuffles order every epoch
    num_workers=1
)

tot_count = 0
for x, y, y_count in loader:
    tot_count += y_count.sum().item()
tot_count


In [ ]:
config = TrainConfig(
    lr=lr,
    weight_decay=weight_decay,
    epochs=epochs,
    device=device
)

In [ ]:
# reduce training data to see if loss hits zero
dataset.intervals = dataset.intervals[:100]

In [ ]:
model = train_cnn_regressor(
    dataset=dataset,
    batch_size=batch_size,
    config=config,
)

In [ ]:
import torch
import time
from pathlib import Path

date = time.strftime("%Y%m%d_%H")
output_model_path = Path(f'/Users/allywang/Desktop/work/S2026/models/seq_bpnet_{date}.pt')

save_model(model, output_model_path)
print(f"Saved trained model (config + state_dict) to: {output_model_path}")


In [ ]:
output_model_path = '../models/seq_6HdTAG_20260129_14.pt'
model = load_model(output_model_path, device='cpu')

<All keys matched successfully>

In [ ]:
from torch.utils.data import DataLoader

loader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=False,  # shuffles order every epoch
    num_workers=1
)


In [ ]:
visualize_split_predictions(
    model,
    device=device,
    n_examples_per_split=10,
    # loader=loader
)